In [1]:
from mlscorecheck.core import *

In [4]:
lrp = PositiveLikelihoodRatio()

In [5]:
lrp.equation_polynomial

fp*lrp*p - n*tp

In [19]:
from mlscorecheck.core import prevalence_threshold, diagnostic_odds_ratio, balanced_accuracy, cohens_kappa
import sympy as sp

In [13]:
tp = sp.Symbol('tp')
tn = sp.Symbol('tn')
fp = sp.Symbol('fp')
p = sp.Symbol('p')
n = sp.Symbol('n')

In [25]:
cohens_kappa(tp=tp, tn=tn, p=p, n=n)

(tn + tp)/((n + p)*((tn + tp)*(n + p - tn - tp)/(2*tn*tp - 2*(n - tp)*(p - tn)) + (tn + tp)/(n + p)))

In [24]:
sp.factor(cohens_kappa(tp=tp, tn=tn, p=p, n=n))

-2*(n*p - n*tn - p*tp)/(n**2 + n*tn - n*tp + p**2 - p*tn + p*tp)

In [22]:
sp.solve(cohens_kappa(tp=tp, tn=tn, p=p, n=n), dict=True)

TypeError: Invalid NaN comparison

In [18]:
sp.factor(balanced_accuracy(tp=tp, tn=tn, p=p, n=n))

0.5*(n*tp + p*tn)/(n*p)

In [14]:
diagnostic_odds_ratio(tp=tp, tn=tn, p=p, n=n)

tn*tp/((n - tn)*(p - tp))

In [11]:
prevalence_threshold(tp=tp, fp=fp, p=p, n=n)

(-fp/n + sqrt(fp*tp/(n*p)))/(-fp/n + tp/p)

In [3]:
import numpy as np

In [5]:
1 / np.nan

nan

In [2]:
globals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['', 'from mlscorecheck.core import *', 'globals()'],
 '_oh': {},
 '_dh': [PosixPath('/home/gykovacs/workspaces/mlscorecheck/notebooks/development'),
  PosixPath('/home/gykovacs/workspaces/mlscorecheck/notebooks/development')],
 'In': ['', 'from mlscorecheck.core import *', 'globals()'],
 'Out': {},
 'get_ipython': <bound method InteractiveShell.get_ipython of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x7f96be221990>>,
 'exit': <IPython.core.autocall.ZMQExitAutocall at 0x7f96be222260>,
 'quit': <IPython.core.autocall.ZMQExitAutocall at 0x7f96be222260>,
 'open': <function io.open(file, mode='r', buffering=-1, encoding=None, errors=None, newline=None, closefd=True, opener=None)>,
 '_': '',
 '__': '',
 '___': '',


In [69]:
import sympy
from sympy import Symbol

from mlscorecheck.core import (score_function_set, sqrt)

In [70]:
tp = Symbol('tp')
fp = Symbol('fp')
tn = Symbol('tn')
fn = Symbol('fn')
eps = Symbol('eps')
p = Symbol('p')
n = Symbol('n')
beta = Symbol('beta')

In [71]:
atomic_symbols = {'tp': tp, 
                  'fp': fp, 
                  'tn': tn, 
                  'fn': fn, 
                  'eps': eps, 
                  'p': p, 
                  'n': n,
                  'beta': beta}

In [72]:
score_functions = score_function_set()
#del score_functions['mcc']

In [73]:
score_functions

{'acc': <function mlscorecheck.core._scores.accuracy(*, tp, tn, p, n)>,
 'sens': <function mlscorecheck.core._scores.sensitivity(*, tp, p)>,
 'spec': <function mlscorecheck.core._scores.specificity(*, tn, n)>,
 'npv': <function mlscorecheck.core._scores.negative_predictive_value(*, tn, fn)>,
 'ppv': <function mlscorecheck.core._scores.positive_predictive_value(*, tp, fp)>,
 'f_beta': <function mlscorecheck.core._scores.f_beta(*, tp, fp, p, beta=1)>,
 'fm': <function mlscorecheck.core._scores.fowlkes_mallows_index(*, tp, fp, p)>,
 'bm': <function mlscorecheck.core._scores.informedness(*, tp, tn, p, n)>,
 'pt': <function mlscorecheck.core._scores.prevalence_threshold(*, tp, fp, p, n)>,
 'lrp': <function mlscorecheck.core._scores.positive_likelihood_ratio(*, tp, fp, p, n)>,
 'lrn': <function mlscorecheck.core._scores.negative_predictive_value(*, tn, fn)>,
 'mk': <function mlscorecheck.core._scores.markedness(*, tp, tn, fp, fn)>,
 'dor': <function mlscorecheck.core._scores.diagnostic_odds_

In [74]:
score_symbols = {key: Symbol(key) for key in score_functions}

In [75]:
score_symbols

{'acc': acc,
 'sens': sens,
 'spec': spec,
 'npv': npv,
 'ppv': ppv,
 'f_beta': f_beta,
 'fm': fm,
 'bm': bm,
 'pt': pt,
 'lrp': lrp,
 'lrn': lrn,
 'mk': mk,
 'dor': dor,
 'mcc': mcc,
 'ji': ji,
 'ba': ba}

In [76]:
score_equations = {}

for key in score_symbols:
    function = score_functions[key]
    params = function.__code__.co_varnames[:function.__code__.co_kwonlyargcount]
    params = {param: atomic_symbols[param] for param in params}
    score_equations[key] = score_symbols[key] - score_functions[key](**params)

In [77]:
score_equations

{'acc': acc - (tn + tp)/(n + p),
 'sens': sens - tp/p,
 'spec': spec - tn/n,
 'npv': npv - tn/(fn + tn),
 'ppv': ppv - tp/(fp + tp),
 'f_beta': f_beta - tp*(beta**2 + 1)/(beta**2*p + fp + tp),
 'fm': fm - sqrt(tp**2/(p*(fp + tp))),
 'bm': bm + 1 - tp/p - tn/n,
 'pt': pt - (-fp/n + sqrt(fp*tp/(n*p)))/(-fp/n + tp/p),
 'lrp': lrp - n*tp/(fp*p),
 'lrn': lrn - tn/(fn + tn),
 'mk': mk - tn/(fn + tn) - tp/(fp + tp) + 1,
 'dor': dor - tn*tp/(fn*fp),
 'mcc': mcc - (-fn*fp + tn*tp)/sqrt(n*p*(fn + tn)*(fp + tp)),
 'ji': ji - tp/(fp + p),
 'ba': ba - 0.5*tp/p - 0.5*tn/n}

In [78]:
tp = Symbol('tp')
fp = Symbol('fp')
tn = Symbol('tn')
fn = Symbol('fn')
eps = Symbol('eps')
p = Symbol('p')
n = Symbol('n')
beta = Symbol('beta')

In [79]:
score_symbols = {key: Symbol(key) for key in score_functions}

In [80]:
score_equations = {}

for key in score_symbols:
    function = score_functions[key]
    params = function.__code__.co_varnames[:function.__code__.co_kwonlyargcount]
    params = {param: atomic_symbols[param] for param in params}
    score_equations[key] = score_symbols[key] - score_functions[key](**params)

In [81]:
eq0 = tp**2 - score_symbols['fm']**2*p*tp - score_symbols['fm']**2*fp*p
eq1 = score_symbols['mk']*(tp + fp)*(tn + fn) - tp*(tn + fn) - tn*(tp + fp) + (tn+fn)*(tp+fp)

In [82]:
eq2 = p - tp - fn
eq3 = n - tn - fp

In [103]:
fps0 = sympy.solve(eq0, fp)
fps1 = sympy.solve(eq1, fp)
fps2 = sympy.solve(eq3, fp)

In [104]:
fps0

[-tp + tp**2/(fm**2*p)]

In [105]:
fps1

[tp*(-fn*mk - mk*tn + tn)/(fn*mk + fn + mk*tn)]

In [106]:
fps2

[n - tn]

In [107]:
eq10 = fps0[0] - fps2[0]
eq11 = fps1[0] - fps2[0]

In [108]:
eq10

-n + tn - tp + tp**2/(fm**2*p)

In [109]:
eq11

-n + tn + tp*(-fn*mk - mk*tn + tn)/(fn*mk + fn + mk*tn)

In [110]:
tns0 = sympy.solve(eq10, tp)
tns1 = sympy.solve(eq11, tp)
tns2 = sympy.solve(eq2, tp)

In [111]:
tns0

[fm*(fm*p - sqrt(p*(fm**2*p + 4*n - 4*tn)))/2,
 fm*(fm*p + sqrt(p*(fm**2*p + 4*n - 4*tn)))/2]

In [112]:
tns1

[(-fn*mk*n + fn*mk*tn - fn*n + fn*tn - mk*n*tn + mk*tn**2)/(fn*mk + mk*tn - tn)]

In [113]:
tns2

[-fn + p]

In [115]:
eq20 = tns0[0] - tns2[0]
eq21 = tns1[0] - tns2[0]

In [116]:
eq20

fm*(fm*p - sqrt(p*(fm**2*p + 4*n - 4*tn)))/2 + fn - p

In [117]:
eq21

fn - p + (-fn*mk*n + fn*mk*tn - fn*n + fn*tn - mk*n*tn + mk*tn**2)/(fn*mk + mk*tn - tn)

In [118]:
tns0 = sympy.solve(eq20, tn)
tns1 = sympy.solve(eq21, tn)

In [119]:
tns0

[fm**2*p/4 + n - (fm**2*p + 2*fn - 2*p)**2/(4*fm**2*p)]

In [120]:
tns1

[(-2*fn*mk + mk*n + mk*p - p - sqrt(4*fn*mk*n + 4*fn*mk*p + mk**2*n**2 + 2*mk**2*n*p + mk**2*p**2 - 2*mk*n*p - 2*mk*p**2 + p**2))/(2*mk),
 (-2*fn*mk + mk*n + mk*p - p + sqrt(4*fn*mk*n + 4*fn*mk*p + mk**2*n**2 + 2*mk**2*n*p + mk**2*p**2 - 2*mk*n*p - 2*mk*p**2 + p**2))/(2*mk)]

In [121]:
eq_final = tns0[0] - tns1[0]

In [124]:
eq_final

fm**2*p/4 + n - (-2*fn*mk + mk*n + mk*p - p - sqrt(4*fn*mk*n + 4*fn*mk*p + mk**2*n**2 + 2*mk**2*n*p + mk**2*p**2 - 2*mk*n*p - 2*mk*p**2 + p**2))/(2*mk) - (fm**2*p + 2*fn - 2*p)**2/(4*fm**2*p)

In [128]:
eq_final = sympy.simplify(eq_final * 2*score_symbols['mk'] * 4 * score_symbols['fm']**2 * p)

In [129]:
eq_final

8*mk*(fm**2*mk*n*p + fm**2*mk*p**2 + fm**2*p**2 + fm**2*p*sqrt(4*fn*mk*n + 4*fn*mk*p + mk**2*n**2 + 2*mk**2*n*p + mk**2*p**2 - 2*mk*n*p - 2*mk*p**2 + p**2) - 2*fn**2*mk + 4*fn*mk*p - 2*mk*p**2)

In [144]:
sympy.solve(sympy.expand(eq_final), fn)

KeyboardInterrupt: 

In [134]:
tmp = sympy.Eq(eq_final, 0)

Eq(8*mk*(fm**2*mk*n*p + fm**2*mk*p**2 + fm**2*p**2 + fm**2*p*sqrt(4*fn*mk*n + 4*fn*mk*p + mk**2*n**2 + 2*mk**2*n*p + mk**2*p**2 - 2*mk*n*p - 2*mk*p**2 + p**2) - 2*fn**2*mk + 4*fn*mk*p - 2*mk*p**2), 0)

In [136]:
expr = 8*score_symbols['mk']*score_symbols['fm']**2*p*sympy.sqrt(4*fn*score_symbols['mk']*n + 4*fn*score_symbols['mk']*p + score_symbols['mk']**2*n**2 + 2*score_symbols['mk']**2*n*p + score_symbols['mk']**2*p**2 - 2*score_symbols['mk']*n*p - 2*score_symbols['mk']*p**2 + p**2)

In [141]:
eq_final2 = sympy.simplify(eq_final - expr)**2 - expr**2

In [146]:
fn_solution = sympy.solve(eq_final2, fn)

In [150]:
fn_solution

[p,
 p - (9*p**2 - 3*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/mk)/(3*(-27*p**3 + sqrt(-4*(9*p**2 - 3*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/mk)**3 + (-54*p**3 + 27*p*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/mk + 27*(-fm**4*n*p**2 - fm**4*p**3 + fm**2*mk*n*p**2 + fm**2*mk*p**3 + fm**2*p**3 - mk*p**3)/mk)**2)/2 + 27*p*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/(2*mk) + 27*(-fm**4*n*p**2 - fm**4*p**3 + fm**2*mk*n*p**2 + fm**2*mk*p**3 + fm**2*p**3 - mk*p**3)/(2*mk))**(1/3)) - (-27*p**3 + sqrt(-4*(9*p**2 - 3*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/mk)**3 + (-54*p**3 + 27*p*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/mk + 27*(-fm**4*n*p**2 - fm**4*p**3 + fm**2*mk*n*p**2 + fm**2*mk*p**3 + fm**2*p**3 - mk*p**3)/mk)**2)/2 + 27*p*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/(2*mk) + 27*(-fm**4*n*p**2 - fm**4*p**3 + fm**2*mk*n*p**2 + fm**2*mk*p**3 + fm**2*p**3 - mk*p**3)/(2*mk))**(1/3)/3,
 p 

In [159]:
sympy.simplify(fn_solution[1])

-2**(1/3)*3**(2/3)*fm**2*n*p/(3*(-9*fm**4*n*p**2/mk - 9*fm**4*p**3/mk + sqrt(3)*sqrt(27*fm**8*p**4*(n + p)**2/mk**2 - 4*fm**6*p**3*(mk*n + mk*p + p)**3/mk**3))**(1/3)) - 2**(1/3)*3**(2/3)*fm**2*p**2/(3*(-9*fm**4*n*p**2/mk - 9*fm**4*p**3/mk + sqrt(3)*sqrt(27*fm**8*p**4*(n + p)**2/mk**2 - 4*fm**6*p**3*(mk*n + mk*p + p)**3/mk**3))**(1/3)) - 2**(1/3)*3**(2/3)*fm**2*p**2/(3*mk*(-9*fm**4*n*p**2/mk - 9*fm**4*p**3/mk + sqrt(3)*sqrt(27*fm**8*p**4*(n + p)**2/mk**2 - 4*fm**6*p**3*(mk*n + mk*p + p)**3/mk**3))**(1/3)) + p - 12**(1/3)*(-9*fm**4*n*p**2/mk - 9*fm**4*p**3/mk + sqrt(3)*sqrt(27*fm**8*p**4*(n + p)**2/mk**2 - 4*fm**6*p**3*(mk*n + mk*p + p)**3/mk**3))**(1/3)/6

In [151]:
tn_ = 10
tp_ = 20
p_ = 40
n_ = 50
fp_ = n_ - tn_
fn_ = p_ - tp_
mk_ = score_functions['mk'](tn=tn_, fn=fn_, tp=tp_, fp=fp_)
fm_ = score_functions['fm'](tp=tp_, fp=fp_, p=p_)

In [153]:
fn_

20

In [156]:
fn_solution[1].subs({p: p_, n: n_, score_symbols['mk']: mk_, score_symbols['fm']: fm_})

20.0000000000000

In [132]:
sympy.solve(eq_final, fn)

KeyboardInterrupt: 

In [125]:
sympy.solve(eq_final, fn)

KeyboardInterrupt: 

In [83]:
tps0 = sympy.solve(eq0, tp)

In [84]:
tps1 = sympy.solve(eq1, tp)

In [85]:
tps2 = sympy.solve(eq2, tp)

In [86]:
tps0

[fm*(fm*p - sqrt(p*(fm**2*p + 4*fp)))/2,
 fm*(fm*p + sqrt(p*(fm**2*p + 4*fp)))/2]

In [87]:
tps1

[fp*(-fn*mk - fn - mk*tn)/(fn*mk + mk*tn - tn)]

In [102]:
tps2

[-fn + p]

In [88]:
eq10 = tps0[0] - tps1[0]
eq11 = tps0[0] - tps2[0]

In [89]:
eq10

fm*(fm*p - sqrt(p*(fm**2*p + 4*fp)))/2 - fp*(-fn*mk - fn - mk*tn)/(fn*mk + mk*tn - tn)

In [90]:
eq11

fm*(fm*p - sqrt(p*(fm**2*p + 4*fp)))/2 + fn - p

In [91]:
eq3

-fp + n - tn

In [93]:
eq2

-fn + p - tp

In [94]:
fns0 = sympy.solve(eq10, fn)

In [96]:
fns0[0]

tn*(-fm**2*mk*p + fm**2*p + fm*mk*sqrt(p*(fm**2*p + 4*fp)) - fm*sqrt(p*(fm**2*p + 4*fp)) - 2*fp*mk)/(fm**2*mk*p - fm*mk*sqrt(p*(fm**2*p + 4*fp)) + 2*fp*mk + 2*fp)

In [98]:
fns1 = sympy.solve(eq2, fn)

In [100]:
eq_fp = fns1[0] - fns0[0]

In [101]:
eq_fp

p - tn*(-fm**2*mk*p + fm**2*p + fm*mk*sqrt(p*(fm**2*p + 4*fp)) - fm*sqrt(p*(fm**2*p + 4*fp)) - 2*fp*mk)/(fm**2*mk*p - fm*mk*sqrt(p*(fm**2*p + 4*fp)) + 2*fp*mk + 2*fp) - tp

In [44]:
fps0 = sympy.solve(eq10, fp)

In [45]:
fps1 = sympy.solve(eq11, fp)

In [46]:
fps2 = sympy.solve(eq3, fp)

In [47]:
fps0

[0,
 fm**2*p*(-fn**2*mk - 2*fn*mk*tn + fn*tn - mk*tn**2 + tn**2)/(fn**2*mk**2 + 2*fn**2*mk + fn**2 + 2*fn*mk**2*tn + 2*fn*mk*tn + mk**2*tn**2)]

In [48]:
fps1

[(-fm**4*p**2 + (fm**2*p + 2*fn - 2*p)**2)/(4*fm**2*p)]

In [49]:
fps2

[n - tn]

In [50]:
eq20 = fps0[1] - fps1[0]
eq21 = fps0[1] - fps2[0]

In [51]:
eq20

fm**2*p*(-fn**2*mk - 2*fn*mk*tn + fn*tn - mk*tn**2 + tn**2)/(fn**2*mk**2 + 2*fn**2*mk + fn**2 + 2*fn*mk**2*tn + 2*fn*mk*tn + mk**2*tn**2) - (-fm**4*p**2 + (fm**2*p + 2*fn - 2*p)**2)/(4*fm**2*p)

In [52]:
eq21

fm**2*p*(-fn**2*mk - 2*fn*mk*tn + fn*tn - mk*tn**2 + tn**2)/(fn**2*mk**2 + 2*fn**2*mk + fn**2 + 2*fn*mk**2*tn + 2*fn*mk*tn + mk**2*tn**2) - n + tn

In [55]:
fns0 = sympy.solve(eq20, fn)

In [56]:
fns1 = sympy.solve(eq21, fn)

In [57]:
fns0

[(-fm**2*p + mk*p - mk*tn + p - sqrt(fm**4*p**2 - 2*fm**2*mk*p**2 - 2*fm**2*mk*p*tn - 2*fm**2*p**2 - 4*fm**2*p*tn + mk**2*p**2 + 2*mk**2*p*tn + mk**2*tn**2 + 2*mk*p**2 + 2*mk*p*tn + p**2))/(2*mk + 2),
 (-fm**2*p + mk*p - mk*tn + p + sqrt(fm**4*p**2 - 2*fm**2*mk*p**2 - 2*fm**2*mk*p*tn - 2*fm**2*p**2 - 4*fm**2*p*tn + mk**2*p**2 + 2*mk**2*p*tn + mk**2*tn**2 + 2*mk*p**2 + 2*mk*p*tn + p**2))/(2*mk + 2),
 (-fm**2*mk*p + mk*p - mk*tn + p - sqrt(fm**4*mk**2*p**2 - 2*fm**2*mk**2*p**2 - 2*fm**2*mk**2*p*tn - 2*fm**2*mk*p**2 + 4*fm**2*p*tn + mk**2*p**2 + 2*mk**2*p*tn + mk**2*tn**2 + 2*mk*p**2 + 2*mk*p*tn + p**2))/(2*mk + 2),
 (-fm**2*mk*p + mk*p - mk*tn + p + sqrt(fm**4*mk**2*p**2 - 2*fm**2*mk**2*p**2 - 2*fm**2*mk**2*p*tn - 2*fm**2*mk*p**2 + 4*fm**2*p*tn + mk**2*p**2 + 2*mk**2*p*tn + mk**2*tn**2 + 2*mk*p**2 + 2*mk*p*tn + p**2))/(2*mk + 2)]

In [58]:
fns1

[tn*(-2*fm**2*mk*p + fm**2*p - fm*sqrt(p*(fm**2*p + 4*n - 4*tn)) - 2*mk**2*n + 2*mk**2*tn - 2*mk*n + 2*mk*tn)/(2*(fm**2*mk*p + mk**2*n - mk**2*tn + 2*mk*n - 2*mk*tn + n - tn)),
 tn*(-2*fm**2*mk*p + fm**2*p + fm*sqrt(p*(fm**2*p + 4*n - 4*tn)) - 2*mk**2*n + 2*mk**2*tn - 2*mk*n + 2*mk*tn)/(2*(fm**2*mk*p + mk**2*n - mk**2*tn + 2*mk*n - 2*mk*tn + n - tn))]

In [60]:
eq30 = fns0[0] - fns1[0]

In [61]:
sympy.simplify(eq30)

(tn*(mk + 1)*(2*fm**2*mk*p - fm**2*p + fm*sqrt(p*(fm**2*p + 4*n - 4*tn)) + 2*mk**2*n - 2*mk**2*tn + 2*mk*n - 2*mk*tn) + (-fm**2*p + mk*p - mk*tn + p - sqrt(fm**4*p**2 - 2*fm**2*mk*p**2 - 2*fm**2*mk*p*tn - 2*fm**2*p**2 - 4*fm**2*p*tn + mk**2*p**2 + 2*mk**2*p*tn + mk**2*tn**2 + 2*mk*p**2 + 2*mk*p*tn + p**2))*(fm**2*mk*p + mk**2*n - mk**2*tn + 2*mk*n - 2*mk*tn + n - tn))/(2*(mk + 1)*(fm**2*mk*p + mk**2*n - mk**2*tn + 2*mk*n - 2*mk*tn + n - tn))

In [62]:
sympy.factor(eq30)

-(fm**4*mk*p**2 + fm**2*mk**2*n*p - fm**2*mk**2*p**2 - 2*fm**2*mk**2*p*tn + 2*fm**2*mk*n*p - fm**2*mk*p**2 - 3*fm**2*mk*p*tn + fm**2*mk*p*sqrt(fm**4*p**2 - 2*fm**2*mk*p**2 - 2*fm**2*mk*p*tn - 2*fm**2*p**2 - 4*fm**2*p*tn + mk**2*p**2 + 2*mk**2*p*tn + mk**2*tn**2 + 2*mk*p**2 + 2*mk*p*tn + p**2) + fm**2*n*p - fm*mk*tn*sqrt(fm**2*p**2 + 4*n*p - 4*p*tn) - fm*tn*sqrt(fm**2*p**2 + 4*n*p - 4*p*tn) - mk**3*n*p - mk**3*n*tn + mk**3*p*tn + mk**3*tn**2 - 3*mk**2*n*p - 2*mk**2*n*tn + mk**2*n*sqrt(fm**4*p**2 - 2*fm**2*mk*p**2 - 2*fm**2*mk*p*tn - 2*fm**2*p**2 - 4*fm**2*p*tn + mk**2*p**2 + 2*mk**2*p*tn + mk**2*tn**2 + 2*mk*p**2 + 2*mk*p*tn + p**2) + 3*mk**2*p*tn + 2*mk**2*tn**2 - mk**2*tn*sqrt(fm**4*p**2 - 2*fm**2*mk*p**2 - 2*fm**2*mk*p*tn - 2*fm**2*p**2 - 4*fm**2*p*tn + mk**2*p**2 + 2*mk**2*p*tn + mk**2*tn**2 + 2*mk*p**2 + 2*mk*p*tn + p**2) - 3*mk*n*p - mk*n*tn + 2*mk*n*sqrt(fm**4*p**2 - 2*fm**2*mk*p**2 - 2*fm**2*mk*p*tn - 2*fm**2*p**2 - 4*fm**2*p*tn + mk**2*p**2 + 2*mk**2*p*tn + mk**2*tn**2 + 2*mk*p

In [66]:
eq30num, eq30den = sympy.fraction(sympy.factor(eq30))

In [67]:
eq30num

-fm**4*mk*p**2 - fm**2*mk**2*n*p + fm**2*mk**2*p**2 + 2*fm**2*mk**2*p*tn - 2*fm**2*mk*n*p + fm**2*mk*p**2 + 3*fm**2*mk*p*tn - fm**2*mk*p*sqrt(fm**4*p**2 - 2*fm**2*mk*p**2 - 2*fm**2*mk*p*tn - 2*fm**2*p**2 - 4*fm**2*p*tn + mk**2*p**2 + 2*mk**2*p*tn + mk**2*tn**2 + 2*mk*p**2 + 2*mk*p*tn + p**2) - fm**2*n*p + fm*mk*tn*sqrt(fm**2*p**2 + 4*n*p - 4*p*tn) + fm*tn*sqrt(fm**2*p**2 + 4*n*p - 4*p*tn) + mk**3*n*p + mk**3*n*tn - mk**3*p*tn - mk**3*tn**2 + 3*mk**2*n*p + 2*mk**2*n*tn - mk**2*n*sqrt(fm**4*p**2 - 2*fm**2*mk*p**2 - 2*fm**2*mk*p*tn - 2*fm**2*p**2 - 4*fm**2*p*tn + mk**2*p**2 + 2*mk**2*p*tn + mk**2*tn**2 + 2*mk*p**2 + 2*mk*p*tn + p**2) - 3*mk**2*p*tn - 2*mk**2*tn**2 + mk**2*tn*sqrt(fm**4*p**2 - 2*fm**2*mk*p**2 - 2*fm**2*mk*p*tn - 2*fm**2*p**2 - 4*fm**2*p*tn + mk**2*p**2 + 2*mk**2*p*tn + mk**2*tn**2 + 2*mk*p**2 + 2*mk*p*tn + p**2) + 3*mk*n*p + mk*n*tn - 2*mk*n*sqrt(fm**4*p**2 - 2*fm**2*mk*p**2 - 2*fm**2*mk*p*tn - 2*fm**2*p**2 - 4*fm**2*p*tn + mk**2*p**2 + 2*mk**2*p*tn + mk**2*tn**2 + 2*mk*p*

In [68]:
sympy.solve(sympy.factor(eq30num), tn)

KeyboardInterrupt: 